In [4]:
%pip install plotly
%pip install nbformat
%pip install pandas
%pip install boto3
%pip install matplotlib
%pip install python-dotenv
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.0 MB 17.1 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 13.5 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 13.6 MB/s eta 0:00:01
   -------------------------------------- - 10.5/11.0 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   --- ------------------------------------ 3.4/44.5 MB 16.7 MB/s eta 0:00:03
   ----- ---------------------------------- 5.8/44.5 MB 13.8 MB/s eta 0:00:03
   ------- -------------------------------- 8.1/44.5 MB 13.3 MB/s eta 0:00:03
   --------- ------------------------------ 10.7/44.5 MB 13.1 MB/s eta 0:00:03
   ------------ --------------------------- 13.6/44.5 MB 13.1 MB/s eta 0:00:03
   -------------- ------------------------- 16.3/44.5 MB 13.1 MB/s eta 0:00:


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# --- Celda 1: Importación de librerías ---
import boto3
import pandas as pd
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from dotenv import load_dotenv
import os


In [9]:
# Configurar la conexión con S3 usando variables de entorno
load_dotenv()
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('Access_key_ID'),
    aws_secret_access_key=os.getenv('Secret_access_key')
)

bucket_name = 'chicago-inspections-analytics'
ingesta_inicial_key = 'datos_limpios/datos_limpios/datos_limpios_2024-11-13_fe3e1ed7a40a26cef96cae50369e0e13.pkl'  # Ruta del archivo en S3

# Función para cargar el archivo de ingesta inicial desde S3
def cargar_ingesta_inicial():
    try:
        # Descargar el archivo de S3
        file_obj = s3_client.get_object(Bucket=bucket_name, Key=ingesta_inicial_key)
        file_data = file_obj['Body'].read()
        
        # Convertir el archivo de Pickle a un DataFrame
        df = pickle.loads(file_data)
        print("Ingesta inicial cargada exitosamente.")
        return df
    except Exception as e:
        print("Error al cargar la ingesta inicial desde S3:", e)
        return None

# Cargar el DataFrame de ingesta inicial
df_inicial = cargar_ingesta_inicial()

# Obtener la fecha más reciente
if df_inicial is not None:
    # Asegurarse de que la columna de fecha esté en el formato datetime
    df_inicial['inspection_date'] = pd.to_datetime(df_inicial['inspection_date'], errors='coerce')

    # Obtener la fecha más reciente
    fecha_mas_reciente = df_inicial['inspection_date'].max()
    print("Fecha más reciente en el DataFrame:", fecha_mas_reciente)
else:
    print("No se pudo cargar el DataFrame de ingesta inicial.")

Error al cargar la ingesta inicial desde S3: name 'pickle' is not defined
No se pudo cargar el DataFrame de ingesta inicial.


In [ ]:
# --- Celda 3: Eliminación de observaciones específicas ---
df=df_inicial
exclude_results = ['Business Not Located', 'No Entry', 'Out of Business']
df = df[~df['results'].isin(exclude_results)]

In [ ]:
# --- Celda 4: Transformación de resultados de inspección ---
df['results'] = df['results'].replace({
    'Pass': 'pass',
    'Pass w/ Conditions': 'pass'
}).apply(lambda x: 'fail' if x not in ['pass'] else x)

In [ ]:
# --- Celda 5: Transformación de la variable risk ---
df['risk'] = df['risk'].replace({
    'Risk 1 (High)': 'high',
    'Risk 2 (Medium)': 'medium',
    'Risk 3 (Low)': 'low',
    'All': 'all'
})

In [ ]:
# --- Celda 6: Transformación de la variable facility_type ---
df['facility_type'] = df['facility_type'].replace({
    '.*Daycare.*': 'daycare',
    '.*Restaurant.*': 'restaurant',
    '.*Mobile Food.*': 'mobile food'
}, regex=True)

In [ ]:
# Cambiar facility_type no populares a "other"
top_20 = df['facility_type'].value_counts().nlargest(20).index
df['facility_type'] = df['facility_type'].apply(
    lambda x: x if x in top_20 else 'other'
)

In [ ]:
# --- Celda 7: Generación de características adicionales ---
df['inspection_date'] = pd.to_datetime(df['inspection_date'])
df['month'] = df['inspection_date'].dt.month
df['year'] = df['inspection_date'].dt.year
df['day_of_month'] = df['inspection_date'].dt.day
df['week_of_year'] = df['inspection_date'].dt.isocalendar().week
df['day_of_week'] = df['inspection_date'].dt.dayofweek
df['week_day'] = df['day_of_week'].apply(lambda x: 1 if x < 5 else 0)
df['weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
# --- Celda 8: Creación de la matriz de diseño ---
features = ['facility_type', 'risk', 'latitude', 'longitude', 'results', 
            'month', 'year', 'day_of_month', 'week_of_year', 'week_day', 
            'weekend', 'day_of_week']

df_features = df[features]
column_transformer = ColumnTransformer(transformers=[
    ('facility_type', OneHotEncoder(), ['facility_type']),
    ('risk', OneHotEncoder(), ['risk'])
], remainder='passthrough')

feature_matrix = column_transformer.fit_transform(df_features)

# Convertir a DataFrame
feature_matrix_df = pd.DataFrame(feature_matrix, columns=column_transformer.get_feature_names_out())

In [ ]:
# --- Celda 9: Guardar la matriz de diseño en S3 ---
today_date = datetime.today().strftime('%Y-%m-%d')
file_name = f"feature-matrix/feature-matrix.{today_date}.pkl"

# Guardar temporalmente el archivo en el disco local
with open('/tmp/temp_feature_matrix.pkl', 'wb') as temp_file:
    feature_matrix_df.to_pickle(temp_file)
    s3_client.upload_file('/tmp/temp_feature_matrix.pkl', bucket_name, file_name)

print("Matriz de diseño creada y guardada exitosamente.")